In [113]:
pip install yfinance riskfolio-lib


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [114]:
import yfinance as yf
import pandas as pd
import riskfolio as rp
import matplotlib.pyplot as plt

In [115]:
import yfinance as yf
import riskfolio as rp
import pandas as pd
import numpy as np

# Fetch stock data
tickers = ['NVDA', 'SMCI', 'KNSL', 'GCT', 'PDD', 'CRWD', 'AVGO', 'SNOW']
start_date = '2021-01-01'
end_date = '2024-03-20'

data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

# Resample data to monthly frequency
monthly_data = data.resample('M').last()

# Calculate returns
returns = monthly_data.pct_change().dropna()

# Create a portfolio object
port = rp.Portfolio(returns)

# Estimate expected returns and covariance matrix
port.assets_stats(method_mu='hist', method_cov='hist')

# Define the asset bounds and constraints
port.ainequality = None
port.binequality = None
port.lowerbound = 0.05
port.upperbound = 0.5
port.budget = 1

# Optimize the portfolio using ERP
model = 'Classic' # Could be 'Classic' (historical), 'BL' (Black Litterman), or 'FM' (Factor Model)
rm = 'MV' # Risk measure could be 'MV' (variance), 'MAD' (mean absolute deviation), or 'MSV' (semi-variance)
rf = 0.0 # Risk-free rate
b = None # Risk budgeting vector (None for ERP)

w = port.rp_optimization(model=model, rm=rm, rf=rf, b=b)

# Print the optimal weights
print("\nOptimal Weights:")
print(w.T)

[*********************100%%**********************]  8 of 8 completed


Optimal Weights:
            AVGO     CRWD     GCT     KNSL    NVDA      PDD    SMCI     SNOW
weights 15.1064% 12.3810% 5.1841% 22.2631% 8.5259% 13.6484% 4.6213% 18.2697%


In [116]:
print("\nMonthly Stock Returns")
print(returns)


Monthly Stock Returns
Ticker          AVGO      CRWD       GCT      KNSL      NVDA       PDD   
Date                                                                     
2022-09-30 -10.2937%  -9.7476% -30.9898%   0.7256% -19.5534% -12.2300%  \
2022-10-31   5.8805%  -2.1904% -53.7092%  23.3928%  11.1871% -12.3841%   
2022-11-30  17.2105% -27.0161%  57.4786%  -2.1669%  25.4155%  49.6261%   
2022-12-31   2.3166% -10.5057% -22.7951% -15.1488% -13.6442%  -0.5973%   
2023-01-31   4.6286%   0.5794%   2.8119%   6.4699%  33.6869%  20.1471%   
2023-02-28   1.5863%  13.9660% -13.5043%  14.5098%  18.8309% -10.4613%   
2023-03-31   8.7276%  13.7294%  24.5059%  -5.8205%  19.6662% -13.4846%   
2023-04-30  -2.3443% -12.5382% -12.2222%   8.8489%  -0.1008% -10.2108%   
2023-05-31  28.9641%  33.3861%  22.0615%  -7.2210%  36.3436%  -4.1526%   
2023-06-30   7.9325%  -8.2808%   0.1482%  23.5065%  11.8210%   5.8481%   
2023-07-31   3.5991%  10.0701%  29.2899%  -0.4196%  10.4652%  29.9103%   
2023-08-31   2.

In [117]:
# Calculate monthly volatility
monthly_volatility = returns.std()

# Calculate annual volatility
annual_volatility = monthly_volatility * np.sqrt(12)

# Create a DataFrame to store volatility values
volatility_df = pd.DataFrame({'Monthly Volatility': monthly_volatility,
                              'Annual Volatility': annual_volatility})

# Print the volatility DataFrame
print("Volatility:")
print(volatility_df)


Volatility:
        Monthly Volatility  Annual Volatility
Ticker                                       
AVGO               9.6873%           33.5578%
CRWD              14.9919%           51.9333%
GCT               36.7900%          127.4442%
KNSL              13.0827%           45.3196%
NVDA              15.8875%           55.0358%
PDD               18.9735%           65.7262%
SMCI              35.3095%          122.3157%
SNOW              10.0255%           34.7295%
